In [1]:
!pip install pandas requests numpy scikit-learn lxml seaborn -q


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from requests import get
import re
import numpy as np
import unicodedata

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

In [3]:
customers = pd.read_csv("data/customers_dataset.csv")
sellers = pd.read_csv("data/sellers_dataset.csv")
order_items = pd.read_csv("data/order_items_dataset.csv")
order_payments = pd.read_csv("data/order_payments_dataset.csv")
order_reviews = pd.read_csv("data/order_reviews_dataset.csv")
orders = pd.read_csv("data/orders_dataset.csv")
product_translation = pd.read_csv("data/product_category_name_translation.csv")
products = pd.read_csv("data/products_dataset.csv")
geolocations = pd.read_csv("data/geolocation_dataset.csv")

In [4]:
def clean_text(text: str, only_normalize: bool = False) -> str:
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore").decode("utf-8")

    if not only_normalize:
        text = re.sub(r"^[0-9*\.]+", "", text)
        text = re.sub(r"[^\w\s]", "", text)
        text = re.sub(r"\s+", " ", text)
        text = text.strip().lower()

    return text

In [5]:
def get_coordinate(zip_code: int, geolocation_data: pd.DataFrame) -> pd.Series:
    result = geolocation_data[
        geolocation_data["geolocation_zip_code_prefix"] == zip_code
    ][["geolocation_lat", "geolocation_lng"]]

    if not result.empty:
        return result.sample(1).iloc[0]

    zip_diffs = (geolocation_data["geolocation_zip_code_prefix"] - zip_code).abs()
    closest_index = zip_diffs.idxmin()
    closest_zip_code = geolocation_data.loc[
        closest_index, "geolocation_zip_code_prefix"
    ]

    nearby_coords = geolocation_data[
        geolocation_data["geolocation_zip_code_prefix"] == closest_zip_code
    ][["geolocation_lat", "geolocation_lng"]].mean()

    return nearby_coords

## State's Abbreviation


In [6]:
state = pd.read_html(get("https://brazil-help.com/brazilian_states.htm").content)[2]

state.columns = state.iloc[1]

state = state.iloc[2:]

state = state[["Common Two Letter Abbreviation", "State"]]


state.State = state.State.apply(lambda x: clean_text(x, True))

state.sample(5, random_state=42)

1,Common Two Letter Abbreviation,State
10,GO,Goias
15,PA,Para
11,MA,Maranhao
23,RO,Rondonia
2,AC,Acre


In [7]:
customers.replace(dict(state.values), inplace=True)
sellers.replace(dict(state.values), inplace=True)
geolocations.replace(dict(state.values), inplace=True)

geolocations.sample(5, random_state=42)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
162456,6412,-23.497836,-46.882982,barueri,Sao Paulo
65543,3707,-23.507041,-46.543324,sao paulo,Sao Paulo
129566,5406,-23.555752,-46.674463,são paulo,Sao Paulo
999061,99709,-27.661007,-52.285391,erechim,Rio Grande do Sul
564558,31310,-19.869566,-43.986149,belo horizonte,Minas Gerais


## Geolocation


In [8]:
geolocations.sample(5, random_state=42)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
162456,6412,-23.497836,-46.882982,barueri,Sao Paulo
65543,3707,-23.507041,-46.543324,sao paulo,Sao Paulo
129566,5406,-23.555752,-46.674463,são paulo,Sao Paulo
999061,99709,-27.661007,-52.285391,erechim,Rio Grande do Sul
564558,31310,-19.869566,-43.986149,belo horizonte,Minas Gerais


In [9]:
geolocations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


In [10]:
geolocations.isna().sum()

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

In [11]:
geolocations.geolocation_city = geolocations.geolocation_city.apply(
    lambda x: clean_text(x, False)
)

geolocations.sample(5, random_state=42)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
162456,6412,-23.497836,-46.882982,barueri,Sao Paulo
65543,3707,-23.507041,-46.543324,sao paulo,Sao Paulo
129566,5406,-23.555752,-46.674463,sao paulo,Sao Paulo
999061,99709,-27.661007,-52.285391,erechim,Rio Grande do Sul
564558,31310,-19.869566,-43.986149,belo horizonte,Minas Gerais


## Product


In [12]:
products.sample(5, random_state=42)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
24880,f819f0c84a64f02d3a5606ca95edd272,relogios_presentes,59.0,452.0,1.0,710.0,19.0,13.0,14.0
6366,b9de40e43fccb6ba53b7eadbd5c49077,utilidades_domesticas,41.0,1188.0,3.0,700.0,70.0,10.0,15.0
4989,26afe4ed5890d941fcba14205863eec7,moveis_decoracao,47.0,1092.0,1.0,3850.0,30.0,30.0,30.0
8135,986870b9985d95ef6205bae1953a61a5,moveis_decoracao,50.0,646.0,1.0,300.0,16.0,30.0,20.0
19482,28ce57ecf4afba85e6020ec0209cada9,cool_stuff,31.0,248.0,2.0,5650.0,73.0,73.0,20.0


In [13]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [14]:
products.product_category_name = products.product_category_name.replace(
    dict(product_translation.values)
)


products.sample(5, random_state=42)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
24880,f819f0c84a64f02d3a5606ca95edd272,watches_gifts,59.0,452.0,1.0,710.0,19.0,13.0,14.0
6366,b9de40e43fccb6ba53b7eadbd5c49077,housewares,41.0,1188.0,3.0,700.0,70.0,10.0,15.0
4989,26afe4ed5890d941fcba14205863eec7,furniture_decor,47.0,1092.0,1.0,3850.0,30.0,30.0,30.0
8135,986870b9985d95ef6205bae1953a61a5,furniture_decor,50.0,646.0,1.0,300.0,16.0,30.0,20.0
19482,28ce57ecf4afba85e6020ec0209cada9,cool_stuff,31.0,248.0,2.0,5650.0,73.0,73.0,20.0


In [15]:
products.isna().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [16]:
numeric_cols = products.select_dtypes(include="number").columns
categorical_cols = products.select_dtypes(
    include=["object", "category", "string"]
).columns

products[numeric_cols] = products[numeric_cols].fillna(products[numeric_cols].median())
for col in categorical_cols:
    mode_value = products[col].mode()[0]
    products[col] = products[col].fillna(mode_value)


products.sample(5, random_state=42)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
24880,f819f0c84a64f02d3a5606ca95edd272,watches_gifts,59.0,452.0,1.0,710.0,19.0,13.0,14.0
6366,b9de40e43fccb6ba53b7eadbd5c49077,housewares,41.0,1188.0,3.0,700.0,70.0,10.0,15.0
4989,26afe4ed5890d941fcba14205863eec7,furniture_decor,47.0,1092.0,1.0,3850.0,30.0,30.0,30.0
8135,986870b9985d95ef6205bae1953a61a5,furniture_decor,50.0,646.0,1.0,300.0,16.0,30.0,20.0
19482,28ce57ecf4afba85e6020ec0209cada9,cool_stuff,31.0,248.0,2.0,5650.0,73.0,73.0,20.0


In [17]:
products["product_volume_cm3"] = (
    products["product_length_cm"]
    * products["product_height_cm"]
    * products["product_width_cm"]
)
products["product_density"] = products["product_weight_g"] / products[
    "product_volume_cm3"
].replace(0, np.nan)

products = products.drop(
    columns=[
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm",
    ]
)

products.sample(5, random_state=42)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_volume_cm3,product_density
24880,f819f0c84a64f02d3a5606ca95edd272,watches_gifts,59.0,452.0,1.0,3458.0,0.205321
6366,b9de40e43fccb6ba53b7eadbd5c49077,housewares,41.0,1188.0,3.0,10500.0,0.066667
4989,26afe4ed5890d941fcba14205863eec7,furniture_decor,47.0,1092.0,1.0,27000.0,0.142593
8135,986870b9985d95ef6205bae1953a61a5,furniture_decor,50.0,646.0,1.0,9600.0,0.031250
19482,28ce57ecf4afba85e6020ec0209cada9,cool_stuff,31.0,248.0,2.0,106580.0,0.053012


In [18]:
products.isna().sum()

product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_volume_cm3            0
product_density               0
dtype: int64

## Orders


In [19]:
orders.sample(5, random_state=42)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
52263,b9a6c5f5df52c7226ac85aee7524c27f,f160aaf480efdfa7268f0fa535f73e76,delivered,2018-06-12 20:07:44,2018-06-12 20:44:26,2018-06-13 13:09:00,2018-06-19 12:44:08,2018-07-17 00:00:00
46645,261e71d2349c713eafa9f3df5972b95d,d6708bbbd2d419475869a84e41f620a1,delivered,2018-01-20 12:15:57,2018-01-20 12:37:13,2018-01-25 21:42:52,2018-01-30 11:32:35,2018-02-15 00:00:00
37546,67b50899f52995848c427e361e10dde3,1b353c00c71689afba44554e43cc5a76,delivered,2018-06-16 21:24:10,2018-06-16 21:36:59,2018-06-21 13:55:00,2018-06-27 13:17:27,2018-07-16 00:00:00
94756,32733fc014b67ef70fa6039dd8c6ba82,ad66699beab381d0747605f108cc02b4,delivered,2017-08-30 21:12:28,2017-08-31 02:50:24,2017-09-12 20:16:46,2017-09-25 17:53:23,2017-09-22 00:00:00
14771,39a70e9e9b729b11dee34ac12478597f,20c678bfce3c8252288f70dd2bee5e51,delivered,2017-08-10 21:26:25,2017-08-10 21:44:13,2017-08-11 19:11:16,2017-08-22 16:45:00,2017-09-12 00:00:00


In [20]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [21]:
datetime_columns = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]
for i in datetime_columns:
    orders[i] = pd.to_datetime(orders[i])

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [22]:
orders["purchase_dayofweek"] = pd.to_datetime(
    orders["order_purchase_timestamp"]
).dt.dayofweek
orders["purchase_hour"] = pd.to_datetime(orders["order_purchase_timestamp"]).dt.hour

orders["approval_delay_hours"] = (
    pd.to_datetime(orders["order_approved_at"])
    - pd.to_datetime(orders["order_purchase_timestamp"])
).dt.total_seconds() / 3600

orders["shipping_delay_hours"] = (
    pd.to_datetime(orders["order_delivered_carrier_date"])
    - pd.to_datetime(orders["order_approved_at"])
).dt.total_seconds() / 3600

orders["delivery_time_hours"] = (
    pd.to_datetime(orders["order_delivered_customer_date"])
    - pd.to_datetime(orders["order_delivered_carrier_date"])
).dt.total_seconds() / 3600

orders["estimated_vs_actual_delivery_diff"] = (
    pd.to_datetime(orders["order_estimated_delivery_date"])
    - pd.to_datetime(orders["order_delivered_customer_date"])
).dt.days

orders = orders.drop(
    columns=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ]
)

orders.sample(5, random_state=42)

,order_id,customer_id,order_status,purchase_dayofweek,purchase_hour,approval_delay_hours,shipping_delay_hours,delivery_time_hours,estimated_vs_actual_delivery_diff
52263,b9a6c5f5df52c7226ac85aee7524c27f,f160aaf480efdfa7268f0fa535f73e76,delivered,1,20,0.611667,16.409444,143.585556,27.0
46645,261e71d2349c713eafa9f3df5972b95d,d6708bbbd2d419475869a84e41f620a1,delivered,5,12,0.354444,129.094167,109.828611,15.0
37546,67b50899f52995848c427e361e10dde3,1b353c00c71689afba44554e43cc5a76,delivered,5,21,0.213611,112.300278,143.374167,18.0
94756,32733fc014b67ef70fa6039dd8c6ba82,ad66699beab381d0747605f108cc02b4,delivered,2,21,5.632222,305.439444,309.610278,-4.0
14771,39a70e9e9b729b11dee34ac12478597f,20c678bfce3c8252288f70dd2bee5e51,delivered,3,21,0.296667,21.450833,261.562222,20.0


In [23]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   order_id                           99441 non-null  object 
 1   customer_id                        99441 non-null  object 
 2   order_status                       99441 non-null  object 
 3   purchase_dayofweek                 99441 non-null  int32  
 4   purchase_hour                      99441 non-null  int32  
 5   approval_delay_hours               99281 non-null  float64
 6   shipping_delay_hours               97644 non-null  float64
 7   delivery_time_hours                96475 non-null  float64
 8   estimated_vs_actual_delivery_diff  96476 non-null  float64
dtypes: float64(4), int32(2), object(3)
memory usage: 6.1+ MB


## Order Items


In [24]:
order_items.sample(5, random_state=42)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
107777,f4ee4273538924bda6212f5948e80fde,1,69455f41626a745aea9ee9164cb9eafd,7d13fca15225358621be4086e1eb0964,2018-06-19 09:18:15,180.00,20.45
2391,056349f85a73d794119c4286c95a52de,1,af35be35db4ad0dc288b571453337376,d20b021d3efdf267a402c402a48ea64b,2017-03-09 14:35:09,10.99,16.05
77829,b124967afcc82ef17ec41020fe2a9136,1,12e6d0f655986ceff00c74658dec97b1,3be634553519fb6536a03e1358e9fdc7,2018-06-06 17:18:09,49.99,8.88
99819,e257ae8610fb4fb68a1f459c3a4b1f51,1,a50acd33ba7a8da8e9db65094fa990a4,8581055ce74af1daba164fdbd55a40de,2017-05-18 08:02:19,117.30,14.43
41297,5e114d8e3840661abc3d9c4820f427b3,1,5cca3efb9521cc1d7099d610d4a12017,3d871de0142ce09b7081e2b9d1733cb1,2018-05-03 18:15:09,58.90,13.77


In [25]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [26]:
datetime_columns = ["shipping_limit_date"]
for i in datetime_columns:
    order_items[i] = pd.to_datetime(order_items[i])

order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


In [27]:
order_items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

## Order Payments


In [28]:
order_payments.sample(5, random_state=42)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
21856,61c7b1cf28b58654fe411aaf87fe249d,1,debit_card,1,39.60
82913,a01f943f4a95572159f0603bd160d67a,1,credit_card,1,111.42
81375,267968a4442f2e55c2904cac31c26660,1,credit_card,1,37.37
9274,303c1a4cb953c69bf9ad12194dc3d44d,1,credit_card,1,47.96
77826,c5f5087eaff455f93653900411e34432,1,boleto,1,141.88


In [29]:
order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [30]:
order_payments.isna().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

## Order Reviews


In [31]:
order_reviews.sample(5, random_state=42)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
90252,406e32984dd5273582105460a79571af,e00ed9d20c3479f9f0e9727ca9d60946,5,NaN,NaN,2017-12-22 00:00:00,2017-12-22 21:10:19
24436,3bca8d9922bed47eb96f23d121945290,3b5351f5f99b46339212291661a9d226,5,NaN,Cumpriu o acordado!,2018-04-10 00:00:00,2018-04-13 01:57:05
11313,65895b807ac5dfe062c82400b3f210b8,f395e98fb5c1c6ce1306e80de2fe125b,4,NaN,NaN,2017-04-20 00:00:00,2017-04-21 11:31:32
75442,2a6faa65a6e893105c60b1018d40e14a,57899333b5e286632bd2599d3f7864ce,5,NaN,NaN,2018-04-06 00:00:00,2018-04-07 00:34:14
7217,a738aa683a09dc5979abc7d9c2cc8029,0cc76fbe09687fda664178e9fc6c404f,5,NaN,NaN,2018-01-07 00:00:00,2018-01-08 13:48:20


In [32]:
order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [33]:
datetime_columns = [
    "review_creation_date",
    "review_answer_timestamp",
]
for i in datetime_columns:
    order_reviews[i] = pd.to_datetime(order_reviews[i])

order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                99224 non-null  object        
 1   order_id                 99224 non-null  object        
 2   review_score             99224 non-null  int64         
 3   review_comment_title     11568 non-null  object        
 4   review_comment_message   40977 non-null  object        
 5   review_creation_date     99224 non-null  datetime64[ns]
 6   review_answer_timestamp  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 5.3+ MB


In [34]:
order_reviews["review_length"] = (
    order_reviews["review_comment_message"].fillna("").apply(len)
)
order_reviews = order_reviews.drop(
    columns=["review_comment_title", "review_comment_message"]
)

order_reviews.sample(5, random_state=42)

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp,review_length
90252,406e32984dd5273582105460a79571af,e00ed9d20c3479f9f0e9727ca9d60946,5,2017-12-22,2017-12-22 21:10:19,0
24436,3bca8d9922bed47eb96f23d121945290,3b5351f5f99b46339212291661a9d226,5,2018-04-10,2018-04-13 01:57:05,19
11313,65895b807ac5dfe062c82400b3f210b8,f395e98fb5c1c6ce1306e80de2fe125b,4,2017-04-20,2017-04-21 11:31:32,0
75442,2a6faa65a6e893105c60b1018d40e14a,57899333b5e286632bd2599d3f7864ce,5,2018-04-06,2018-04-07 00:34:14,0
7217,a738aa683a09dc5979abc7d9c2cc8029,0cc76fbe09687fda664178e9fc6c404f,5,2018-01-07,2018-01-08 13:48:20,0


In [35]:
order_reviews.isna().sum()

review_id                  0
order_id                   0
review_score               0
review_creation_date       0
review_answer_timestamp    0
review_length              0
dtype: int64

## Customers


In [36]:
customers.sample(5, random_state=42)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
52263,c7432c6d237ffd6aa36a007b4237ec38,9a897ea48bf988012c00b802f1104a92,2971,sao paulo,Sao Paulo
46645,7f399d641e2e2064470145178c9e8778,90436a67885a57f147fb79e6d0e4bc1c,38610,unai,Minas Gerais
37546,ba5642b730704dc0f74b7cf715b41ed5,4d8056f71519ae1069e6747c63c676f7,88820,icara,Santa Catarina
94756,0f346a2cc84ebb2d52f0759d0acfd030,6117c9ef3251089693a6abb90c195eba,25250,duque de caxias,Rio de Janeiro
14771,d393b9491df482cf448e60aa9955b7f2,5caf3a2a5d1ef808e3dd182e79baa392,36955,mutum,Minas Gerais


In [37]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [38]:
customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [39]:
customers[["customer_geolocation_lat", "customer_geolocation_lng"]] = (
    customers.customer_zip_code_prefix.apply(lambda x: get_coordinate(x, geolocations))
)
customers.sample(5, random_state=42)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_geolocation_lat,customer_geolocation_lng
52263,c7432c6d237ffd6aa36a007b4237ec38,9a897ea48bf988012c00b802f1104a92,2971,sao paulo,Sao Paulo,-23.486544,-46.711094
46645,7f399d641e2e2064470145178c9e8778,90436a67885a57f147fb79e6d0e4bc1c,38610,unai,Minas Gerais,-16.362260,-46.902908
37546,ba5642b730704dc0f74b7cf715b41ed5,4d8056f71519ae1069e6747c63c676f7,88820,icara,Santa Catarina,-28.685361,-49.327491
94756,0f346a2cc84ebb2d52f0759d0acfd030,6117c9ef3251089693a6abb90c195eba,25250,duque de caxias,Rio de Janeiro,-22.601466,-43.302854
14771,d393b9491df482cf448e60aa9955b7f2,5caf3a2a5d1ef808e3dd182e79baa392,36955,mutum,Minas Gerais,-19.818436,-41.437648


In [40]:
customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
customer_geolocation_lat    0
customer_geolocation_lng    0
dtype: int64

# Seller


In [41]:
sellers.sample(5, random_state=42)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
1947,1f7fd2a6fcd5a6fa5d8a4dabc72aaae0,95800,venancio aires,Rio Grande do Sul
1309,6e1862e15f33d9994bc25922a85e1efc,13505,rio claro,Sao Paulo
1606,e628d4a53c109f09ca88098338b3a3f5,30170,belo horizonte,Minas Gerais
2626,0249d282d911d23cb8b869ab49c99f53,5676,sao paulo,Sao Paulo
2440,bdae679a9b282249bc23b9b69dae9a99,72210,brasilia,Distrito Federal


In [42]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [43]:
sellers.isna().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

In [44]:
sellers[["seller_geolocation_lat", "seller_geolocation_lng"]] = (
    sellers.seller_zip_code_prefix.apply(lambda x: get_coordinate(x, geolocations))
)
sellers.sample(5, random_state=42)

,seller_id,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng
1947,1f7fd2a6fcd5a6fa5d8a4dabc72aaae0,95800,venancio aires,Rio Grande do Sul,-29.612950,-52.189739
1309,6e1862e15f33d9994bc25922a85e1efc,13505,rio claro,Sao Paulo,-22.382404,-47.567473
1606,e628d4a53c109f09ca88098338b3a3f5,30170,belo horizonte,Minas Gerais,-19.918661,-43.941245
2626,0249d282d911d23cb8b869ab49c99f53,5676,sao paulo,Sao Paulo,-23.601384,-46.697738
2440,bdae679a9b282249bc23b9b69dae9a99,72210,brasilia,Distrito Federal,-15.805735,-48.106373


In [45]:
sellers.isna().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
seller_geolocation_lat    0
seller_geolocation_lng    0
dtype: int64

# Assambling Data


In [46]:
detailed_orders_data = (
    order_items.merge(orders, on="order_id")
    .merge(order_payments, on="order_id")
    .merge(products, on="product_id")
    .merge(customers, on="customer_id")
    .merge(sellers, on="seller_id")
    .merge(order_reviews, on="order_id", how="left")
)

detailed_orders_data

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,purchase_dayofweek,purchase_hour,approval_delay_hours,shipping_delay_hours,delivery_time_hours,estimated_vs_actual_delivery_diff,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_volume_cm3,product_density,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_geolocation_lat,customer_geolocation_lng,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng,review_id,review_score,review_creation_date,review_answer_timestamp,review_length
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2,8,0.775833,152.811389,29.158889,8.0,1,credit_card,2,72.19,cool_stuff,58.0,598.0,4.0,3528.0,0.184240,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,Rio de Janeiro,-21.771661,-41.312119,27277,volta redonda,Sao Paulo,-22.499415,-44.124192,97ca439bc427b48bc1cd7177abe71365,5.0,2017-09-21,2017-09-22 10:57:03,46.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2,10,0.201944,195.496389,193.490000,2.0,1,credit_card,3,259.83,pet_shop,56.0,239.0,2.0,60000.0,0.500000,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,Sao Paulo,-20.216537,-50.926688,3471,sao paulo,Sao Paulo,-23.567347,-46.517745,7b07bacd811c4117b742569b04ce3580,4.0,2017-05-13,2017-05-15 11:34:13,0.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,6,14,0.249722,45.805000,144.707778,13.0,1,credit_card,5,216.87,furniture_decor,59.0,695.0,2.0,14157.0,0.215441,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,Minas Gerais,-19.870095,-44.579807,37564,borda da mata,Minas Gerais,-22.258834,-46.147881,0c5b33dea94867d1ac402749e5438e8b,5.0,2018-01-23,2018-01-23 16:06:31,90.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2,10,0.161944,51.295000,96.077500,5.0,1,credit_card,2,25.78,perfumery,42.0,480.0,1.0,2400.0,0.083333,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,Sao Paulo,-23.061701,-46.641062,14403,franca,Sao Paulo,-20.560144,-47.374886,f4028d019cb58564807486a6aaf33817,4.0,2018-08-15,2018-08-15 16:39:01,0.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,5,13,0.206111,283.598889,318.939444,15.0,1,credit_card,3,218.04,garden_tools,59.0,409.0,1.0,42000.0,0.089286,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,Sao Paulo,-23.248077,-46.825404,87900,loanda,Parana,-22.929162,-53.138231,940144190dcba6351888cafa43f3a3a5,5.0,2017-03-02,2017-03-03 10:54:59,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118302,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,delivered,0,13,38.231944,7.966389,370.794444,7.0,1,boleto,1,343.40,housewares,43.0,1002.0,3.0,53400.0,0.190075,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,Maranhao,-2.479952,-44.213115,88303,itajai,Santa Catarina,-26.919547,-48.679104,9185f849f32d82e216a4e025e0c50f5c,5.0,2018-05-11,2018-05-14 12:53:47,0.0
118303,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aa

In [47]:
import numpy as np


def haversine_vectorized(
    df,
    lat1_col="customer_geolocation_lat",
    lon1_col="customer_geolocation_lng",
    lat2_col="seller_geolocation_lat",
    lon2_col="seller_geolocation_lng",
):
    R = 6371.0

    lat1 = np.radians(df[lat1_col])
    lon1 = np.radians(df[lon1_col])
    lat2 = np.radians(df[lat2_col])
    lon2 = np.radians(df[lon2_col])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return np.round(distance, 2)


detailed_orders_data["seller_customer_distance_km"] = haversine_vectorized(
    detailed_orders_data
)

detailed_orders_data.sample(5, random_state=42)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,purchase_dayofweek,purchase_hour,approval_delay_hours,shipping_delay_hours,delivery_time_hours,estimated_vs_actual_delivery_diff,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_volume_cm3,product_density,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_geolocation_lat,customer_geolocation_lng,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng,review_id,review_score,review_creation_date,review_answer_timestamp,review_length,seller_customer_distance_km
63608,89ba725dafc2f8bce0db73d323a687c2,1,a2a074bf18230a66559fa3ba6977bcad,92eb0f42c21942b6552362b9b114707d,2018-01-19 15:50:52,18.77,16.79,91922c858bb7e396c7116aed07fc63db,delivered,0,15,0.196389,92.554167,440.094444,9.0,1,credit_card,7,71.12,telephony,53.0,1307.0,2.0,756.0,0.220899,1ab7e5facf5738d97496473c68c21f35,78360,campo novo do parecis,MatoGrosso,-13.660345,-57.891216,3504,sao paulo,Sao Paulo,-23.538779,-46.536486,fd5203b3c89f864b99ddb972a9be64e8,5.0,2018-02-07,2018-02-08 11:31:28,58.0,1622.74
95098,cd9f7b10571863a4339b5ccdd2365f2f,1,ed811f9c4ce4994aa221a2b3875666db,2138ccb85b11a4ec1e37afbd1c8eda1f,2018-02-27 16:16:15,24.99,11.85,09a1d8d34896cc9569f3019eb0adec48,delivered,2,16,0.161944,124.469167,595.242778,-11.0,1,credit_card,1,36.84,telephony,45.0,349.0,11.0,627.0,0.159490,18e18ed65f66796b16d0ba5bf0d68844,14030,ribeirao preto,Sao Paulo,-21.185910,-47.830428,8250,sao paulo,Sao Paulo,-23.541239,-46.434155,fabddde29f614fc8914b28f6cc55eba7,2.0,2018-03-15,2018-03-16 12:54:27,0.0,298.67
104868,e2747b7f8ef2d891fc5c127dc44c68b5,1,c56400aa5823575d5a92a8f98077490a,7c67e1448b00f6e969d365cea6b010ab,2017-08-08 22:55:19,142.99,18.72,36a49b54cbbe46996c20730dcc059f30,delivered,3,22,0.236667,189.310833,91.105278,6.0,1,credit_card,10,161.71,office_furniture,48.0,1043.0,1.0,63700.0,0.143642,281e1e72207e518c29b249a1361a775d,9190,santo andre,Sao Paulo,-23.674318,-46.538599,8577,itaquaquecetuba,Sao Paulo,-23.488234,-46.362383,64977cdf05527392ac01d6a3e3c9b439,5.0,2017-08-09,2017-08-10 02:46:59,0.0,27.40
41995,5aa4778f637df7a41e0df0f10ee0f1c1,1,fe95eb948375116da759c8ef60295001,4a3ca9315b744ce9f8e9374361493884,2017-04-10 17:25:12,65.90,11.85,574733d6b5888f34d3918ffe1d629bb3,delivered,1,17,0.142778,46.269444,121.091944,13.0,1,credit_card,3,77.75,bed_bath_table,58.0,556.0,1.0,23625.0,0.033862,2c58adb81726bab4f6d8836e8d6a217f,14040,ribeirao preto,Sao Paulo,-21.175127,-47.865834,14940,ibitinga,Sao Paulo,-21.756869,-48.839401,ae43c1f9d8746587e0f0f4a52aeb8d6b,5.0,2017-04-12,2017-04-17 11:49:29,0.0,119.73
28980,3e9fee97691551e4a1444f095899d5f9,1,0efe972876d70927f5099b77f387c6c7,4a3ca9315b744ce9f8e9374361493884,2017-09-08 12:04:33,83.90,17.84,b3e3fe3971fb3b7c1776e69da97eacea,delivered,4,11,0.252500,99.649167,219.482222,6.0,1,credit_card,2,101.74,furniture_decor,63.0,418.0,1.0,23625.0,0.057143,7898f066e7ca1be625589d012a52db34,22740,rio de janeiro,Rio de Janeiro,-22.928248,-43.362725,14940,ibitinga,Sao Paulo,-21.756869,-48.839401,abe670827aee2d593184e7d9bd68fc90,5.0,2017-09-15,2017-09-18 12:50:34,0.0,578.08


In [48]:
detailed_orders_data.isna().sum()

order_id                                0
order_item_id                           0
product_id                              0
seller_id                               0
shipping_limit_date                     0
price                                   0
freight_value                           0
customer_id                             0
order_status                            0
purchase_dayofweek                      0
purchase_hour                           0
approval_delay_hours                   15
shipping_delay_hours                 1269
delivery_time_hours                  2589
estimated_vs_actual_delivery_diff    2588
payment_sequential                      0
payment_type                            0
payment_installments                    0
payment_value                           0
product_category_name                   0
product_name_lenght                     0
product_description_lenght              0
product_photos_qty                      0
product_volume_cm3                

In [49]:
detailed_orders_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118307 entries, 0 to 118306
Data columns (total 42 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           118307 non-null  object        
 1   order_item_id                      118307 non-null  int64         
 2   product_id                         118307 non-null  object        
 3   seller_id                          118307 non-null  object        
 4   shipping_limit_date                118307 non-null  datetime64[ns]
 5   price                              118307 non-null  float64       
 6   freight_value                      118307 non-null  float64       
 7   customer_id                        118307 non-null  object        
 8   order_status                       118307 non-null  object        
 9   purchase_dayofweek                 118307 non-null  int32         
 10  purchase_hour       

In [50]:
detailed_orders_data.to_csv("data/detailed_orders_dataset.csv", index=False)